In [2]:
import lxml
import newspaper
import mitie

In [3]:
washpost = newspaper.build("https://www.washingtonpost.com/")
atlantic = newspaper.build("https://www.theatlantic.com/")
newyorker = newspaper.build("https://www.newyorker.com/")

In [4]:
articles = []
for i in range(5):
    articles.append(washpost.articles[i])
    articles.append(atlantic.articles[i])
    articles.append(newyorker.articles[i])

In [13]:
texts = []
for article in articles:
    article.download()
    article.parse()
    texts.append(article.text)

In [6]:
import sys, os

In [7]:
sys.path.append(os.getenv('HOME') + '/Packages/MITIE/mitielib')

In [16]:
#words = article.text.split(' ')

In [14]:
tokens = [mitie.tokenize(text) for text in texts]


In [15]:
ner = mitie.named_entity_extractor(os.getenv('HOME') + '/Packages/MITIE/MITIE-models/english/ner_model.dat')

In [19]:
entities_list = [ner.extract_entities(token) for token in tokens]

In [24]:
print(len(entities_list[0]))
print(entities_list[0])

20
[(range(45, 46), 'LOCATION', 0.8384414049273707), (range(58, 60), 'ORGANIZATION', 0.3805900235050664), (range(67, 68), 'LOCATION', 1.259045422024238), (range(155, 156), 'PERSON', 0.3724026272536155), (range(272, 273), 'PERSON', 0.5705829447047538), (range(320, 321), 'LOCATION', 1.5789476063392385), (range(324, 325), 'LOCATION', 1.673072472879782), (range(382, 383), 'LOCATION', 0.6705607291049552), (range(407, 409), 'PERSON', 1.0233118207177625), (range(414, 420), 'ORGANIZATION', 1.4812820063000498), (range(429, 430), 'PERSON', 0.9259654733298314), (range(437, 440), 'ORGANIZATION', 0.4794931683711533), (range(454, 455), 'PERSON', 1.089708498200318), (range(481, 482), 'PERSON', 1.183078487877683), (range(553, 554), 'PERSON', 0.3141024447838465), (range(622, 623), 'PERSON', 0.9916067257667599), (range(670, 671), 'ORGANIZATION', 1.3182720122457192), (range(679, 681), 'PERSON', 1.0738495662525744), (range(688, 690), 'PERSON', 0.3465450793591987), (range(691, 693), 'PERSON', 1.29489097636

In [33]:
entity_texts = []
for index, entities in enumerate(entities_list):
    token = tokens[index]
    temp = []
    for e in entities:
        width = e[0]
        tag = e[1]
        score = e[2]
        score_text = "{:0.3f}".format(score)
        entity_text = " ".join(token[i].decode() for i in width)
        temp.append(entity_text)
        print("   Score: " + score_text + ": " + tag + " " + entity_text)
        
    entity_texts.append(temp)

   Score: 0.838: LOCATION America
   Score: 0.381: ORGANIZATION Generation —
   Score: 1.259: LOCATION U.S.
   Score: 0.372: PERSON Millennials
   Score: 0.571: PERSON Millennials
   Score: 1.579: LOCATION America
   Score: 1.673: LOCATION Washington
   Score: 0.671: LOCATION D.C.
   Score: 1.023: PERSON Michael Lynn
   Score: 1.481: ORGANIZATION Cornell University School of Hotel Administration
   Score: 0.926: PERSON Lynn
   Score: 0.479: ORGANIZATION Restaurants Abandon Tipping
   Score: 1.090: PERSON Lynn
   Score: 1.183: PERSON Lynn
   Score: 0.314: PERSON Millennials
   Score: 0.992: PERSON Lynn
   Score: 1.318: ORGANIZATION IHOP
   Score: 1.074: PERSON Anthony Bourdain
   Score: 0.347: PERSON José Andrés
   Score: 1.295: PERSON Gabrielle Hamilton
   Score: 0.724: PERSON Hacker
   Score: 0.636: PERSON Hacker
   Score: 1.354: LOCATION Nicaragua
   Score: 1.510: LOCATION U.S.
   Score: 1.211: MISC American
   Score: 0.706: PERSON Hacker
   Score: 1.536: LOCATION U.S.
   Score: 0.90

   Score: 0.404: PERSON al-Sham
   Score: 0.355: ORGANIZATION ISIS
   Score: 0.990: LOCATION West
   Score: 0.491: ORGANIZATION Muslim Brotherhood
   Score: 1.260: LOCATION Egypt
   Score: 0.310: ORGANIZATION Islamic State
   Score: 1.067: PERSON David Koresh
   Score: 1.213: PERSON Jim Jones
   Score: 0.702: ORGANIZATION al-Qaeda
   Score: 0.993: PERSON Osama bin Laden
   Score: 0.188: PERSON jihadism
   Score: 0.648: ORGANIZATION al-Qaeda
   Score: 1.343: PERSON Bin Laden
   Score: 0.364: ORGANIZATION Islamic State
   Score: 1.184: PERSON Peter Bergen
   Score: 1.297: PERSON bin Laden
   Score: 0.627: ORGANIZATION Holy War , Inc
   Score: 1.271: PERSON bin Laden
   Score: 1.067: PERSON Bin Laden
   Score: 1.075: LOCATION U.S.
   Score: 1.045: LOCATION Saudi Arabia
   Score: 1.196: PERSON Mohamed Atta
   Score: 0.713: LOCATION Walmart
   Score: 0.299: LOCATION Pizza Hut
   Score: 0.225: ORGANIZATION Islamic State
   Score: 0.854: MISC Western
   Score: 1.104: LOCATION France
   Score:

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
tf = [" ".join(t) for t in entity_texts]

In [62]:
vect = TfidfVectorizer()

In [63]:
tfidf = vect.fit_transform(tf)

In [67]:
terms = vect.get_feature_names()

In [69]:
pairwise_sim = (tfidf * tfidf.T).A

In [48]:
from sklearn.cluster import SpectralClustering

In [91]:
spectral = SpectralClustering(eigen_solver='lobpcg', affinity='precomputed')

In [93]:
spectral.fit(pairwise_sim)

/home/princeaker/.local/lib/python3.5/site-packages/sklearn/manifold/spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


SpectralClustering(affinity='precomputed', assign_labels='kmeans', coef0=1,
          degree=3, eigen_solver='lobpcg', eigen_tol=0.0, gamma=1.0,
          kernel_params=None, n_clusters=8, n_init=10, n_jobs=1,
          n_neighbors=10, random_state=None)

In [78]:
print("Terms per cluster:")
for i in __builtins__.range(8):
    print("Cluster %d:" % i),
    T=t[spectral.labels_==i].indices
    for ind in T:
        print(terms[ind])
    print("")

Terms per cluster:
Cluster 0:


AttributeError: 'numpy.ndarray' object has no attribute 'indices'

In [94]:
spectral.labels_

array([6, 2, 4, 4, 5, 0, 2, 5, 4, 3, 7, 1, 3, 4, 5], dtype=int32)

In [95]:
tfidf[0]

<1x818 sparse matrix of type '<class 'numpy.float64'>'
	with 22 stored elements in Compressed Sparse Row format>